#  SyntaxCutter kasutusnäited

##  Stanza 

In [1]:
# Stanza tagger

import stanza
from estnltk.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

#model_path = ".../estnltk-version_1.6/estnltk/taggers/syntax/stanza_tagger/stanza_resources"
model_path = '/Users/rabauti/stanza-models/stanza_recources'
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)


# SyntaxCutter
from syntaxCutter.cutter import deprelCutter


## Conllu failid 

In [ ]:
#conllu reader
from syntaxCutter.readerСonllu import reader

#kataloog conllu failidega
conllu_folder ='/Users/rabauti/Downloads/conllu_small'

oReader = reader.Reader(path=conllu_folder)

oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)
oCutter.cut(deprel='acl:relcl')


## Andmebaasist 

In [ ]:
from syntaxCutter.readerDB import reader

oReader = reader.Reader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collectionName='koondkorpus_base_subset_of_5000_v2')

oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)

oCutter.cut(deprel='acl:relcl')

## TSV failist

Faili formaat: https://github.com/estnltk/estnltk-model-training/blob/main/collocation_net/data_extraction/from_tsv/collect_texts_db_tsv.ipynb



In [2]:
from syntaxCutter.readerTSV import reader

#subset 5000 TSVfile
TSV_path = '/Users/rabauti/repos/estnltk-model-training/collocation_net/data_extraction/from_tsv/koondkorpus_base_subset_of_5000_v2.tsv'
#full koondkorpus file
TSV_path = '/Volumes/Selena/Kollokatsioonid/koondkorpus/koondkorpus_base_v2_20220216.tsv'
oReader = reader.Reader(file=TSV_path)
oCutter = deprelCutter.Cutter(stanza_tagger=stanza_tagger, Reader=oReader)
oCutter.cut(deprel='acl')

2022-08-15 11:04:51,899 - Cutter - INFO - Start cutting deprel: acl
2022-08-15 11:04:51,903 - Reader - INFO - Reading sentences in progress.


2022-08-15 12:38:16,572 - Reader - INFO - Reading sentences done.
2022-08-15 12:38:16,577 - Cutter - INFO - Done cutting deprel  acl .
2022-08-15 12:38:16,578 - Cutter - INFO - Stats stored in ./result/acl_20220815-110451/stats.tsv .


## Kohandamine

In [ ]:
# Filtreerib korpusest välja ainult hüüumärgiga lõppevad laused. (esialgse morf märgenduse põhjal) 
# Teeb määratud Stanza mudeliga uue analüüsi.
# Ignoreerib lauseid, milles on "conj" märgendiga sõnad.
# Lõikab välja deprel = 'nmod' tipud
# Teeb lühendatud lausele uuesti Stanza mudeliga analüüsi
# Salvestab konserveerunud laused, muutunud laused, laused, millest polnud midagi eemaldada 
# Salvestab üldise statistika


from syntaxCutter.sentence import sentence
class myCustomCutter(deprelCutter.Cutter):

    # funktsioon, kas esialgse süntaksiga lauset filtreerida välja või mitte
    # True puhul lauset ignoreeritakse
    def pre_filter(self, G, deprel):
        #ignoreeri kõiki lauseid, mis ei lõpe hüüumärgiga
        return not (sentence.Sentence.get_prop(G, 'lemma'))[-1:][0] == '!'
        

    # funktsioon, kas defineeritud Stanza mudeliga üle analüüsitud lauset filtreerida välja või mitte
    # True puhul lauset ignoreeritakse
    def filter(self, G, deprel):
        #ignoreeri kõiki lauseid, kus on sees sidesõna
        return  'conj' in sentence.Sentence.get_prop(G, 'deprel')
        #return False

    #oma funktsioon tippude eemaldamiseks
    def cutSentence(self, G, deprel):
        return sentence.Sentence.remove_deprel(G, 'nmod')

TSV_path = '/Users/rabauti/repos/estnltk-model-training/collocation_net/data_extraction/from_tsv/koondkorpus_base_subset_of_5000_v2.tsv'
oReader = reader.Reader(file=TSV_path)

oCutter = myCustomCutter(stanza_tagger=stanza_tagger, Reader=oReader)

oCutter.cut(deprel='mycustomcut')


## TSV fail

Kood TSV faili genereerimiseks asub siin:

https://github.com/estnltk/estnltk-model-training/blob/main/collocation_net/data_extraction/from_tsv/collect_texts_db_tsv.ipynb
